In [1]:
import numpy as np
import pandas as pd
import yaml

In [33]:
def handle_categorical_columns(df):
    """
    The columns "srcAS", "destAS", "srcCC", "destCC" contain only few distinct values.
    It is more efficient to treat them as categorical values and replace each category 
    with an unique integer.
    """
    categorical_columns = ["srcAS", "destAS", "srcCC", "destCC"]
    for column_name in categorical_columns:
        with open(f"mapping_{column_name}.yaml") as f:
            mapping = yaml.safe_load(f)
        print(f"Generating column '{column_name}_cat'")
        df[f"{column_name}_cat"] = df[f"{column_name}"]
        if column_name == "destCC":
            mask = df[f"{column_name}_cat"].isna()
            df.loc[mask, f"{column_name}_cat"] = "  "
        for val, key in zip(mapping.values(), mapping.keys()):
            mask = df[f"{column_name}_cat"] == val
            df.loc[mask, f"{column_name}_cat"] = key
        df[f"{column_name}_cat"] = df[f"{column_name}_cat"].astype(int)
        #df = df.drop([column_name], axis=1)
    return df

def generate_columns_for_sequences(df):
    """
    The columns "lat", "long", "IPSource", "geoCC" and "ASN" contain lists.
    This function creates a column for each item in thos lists. As there are at maximum 39
    values in those lists we need to create 39 additional columns for each of the columns named before.
    If a list is shorter than the maximum value the corresponding columns are filled with default values.
    """
    max_hops = df["num_hops"].max()
    sequence_columns = [("lat", 0.0), ("long", 0.0), ("IPsource", 0), ("geoCC", float("nan")), ("ASN", 0)]
    for column_name, default_value in sequence_columns:
        print(f"Generating new columns for '{column_name}'")
        for i in range(max_hops):
            df[f"{column_name}_{i}"] = default_value
            df[f"{column_name}_{i}"] = df[column_name].apply(lambda x: x[i] if len(x) >= (i+1) else default_value)
            if type(default_value) == int:
                df[f"{column_name}_{i}"] = df[f"{column_name}_{i}"].astype("category")
            # Normalize 'lat' and 'long' columns
            if column_name == "lat":
                df[f"{column_name}_{i}"] = df[f"{column_name}_{i}"] / 90
            if column_name == "long":
                df[f"{column_name}_{i}"] = df[f"{column_name}_{i}"] / 180
        df = df.drop([column_name], axis=1)
    return df

def replace_geoCC_codes_with_integers(df, df_countries):
    """
    The column "geoCC" contains a list of country codes. These should be mapped to integeres.
    """
    max_hops = df["num_hops"].max()
    # Iterate over all geoCC columns
    for i in range(max_hops):
        column_name = f"geoCC_{i}"
        print(f"Replacing strings with category codes in {column_name}")
        for af, af_code in zip(df_countries.index, df_countries["A2Code_cat"]):
            mask = df[column_name] == af
            df.loc[mask, column_name] = af_code + 1 # we have to add 1, because 0 is reserved for nan
        # Replace nan columns with 0
        mask = (df[column_name].isna()) | (df[column_name] == 'nan')
        df.loc[mask, column_name] = 0
        df[column_name] = df[column_name].astype(int)
        df[column_name] = df[column_name].astype("category")
    return df    

In [8]:
# Source: https://www.iban.com/country-codes
df_countries = pd.read_csv("country_codes.tsv", sep='\t')
df_countries["A2Code_cat"] = df_countries["A2Code"].astype("category").cat.codes
df_countries = df_countries.set_index("A2Code")
df_countries

,Country,A3Code,Numeric,A2Code_cat
A2Code,,,,
AF,Afghanistan,AFG,4,2
AL,Albania,ALB,8,5
DZ,Algeria,DZA,12,61
AS,American Samoa,ASM,16,10
AD,Andorra,AND,20,0
...,...,...,...,...
XK,XK,XXX,0,244
YE,Yemen,YEM,887,246
ZM,Zambia,ZMB,894,249


In [9]:
subset = "test"
df = pd.read_pickle(f"dataset_b_noisy_7_agents_{subset}.pkl")

In [34]:
# Prepare data for machine learning, that means transform all columns to numerical values
df = handle_categorical_columns(df)
df = generate_columns_for_sequences(df)
df = replace_geoCC_codes_with_integers(df, df_countries)

Generating column 'srcAS_cat'
Generating column 'destAS_cat'
Generating column 'srcCC_cat'
Generating column 'destCC_cat'
Generating new columns for 'lat'
Generating new columns for 'long'
Generating new columns for 'IPsource'
Generating new columns for 'geoCC'


C:\Users\tilof\AppData\Local\Temp\ipykernel_23784\596585270.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{column_name}_{i}"] = default_value
C:\Users\tilof\AppData\Local\Temp\ipykernel_23784\596585270.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{column_name}_{i}"] = default_value
C:\Users\tilof\AppData\Local\Temp\ipykernel_23784\596585270.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

Generating new columns for 'ASN'


C:\Users\tilof\AppData\Local\Temp\ipykernel_23784\596585270.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{column_name}_{i}"] = default_value
C:\Users\tilof\AppData\Local\Temp\ipykernel_23784\596585270.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{column_name}_{i}"] = default_value
C:\Users\tilof\AppData\Local\Temp\ipykernel_23784\596585270.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

Replacing strings with category codes in geoCC_0
Replacing strings with category codes in geoCC_1
Replacing strings with category codes in geoCC_2
Replacing strings with category codes in geoCC_3
Replacing strings with category codes in geoCC_4
Replacing strings with category codes in geoCC_5
Replacing strings with category codes in geoCC_6
Replacing strings with category codes in geoCC_7
Replacing strings with category codes in geoCC_8
Replacing strings with category codes in geoCC_9
Replacing strings with category codes in geoCC_10
Replacing strings with category codes in geoCC_11
Replacing strings with category codes in geoCC_12
Replacing strings with category codes in geoCC_13
Replacing strings with category codes in geoCC_14
Replacing strings with category codes in geoCC_15
Replacing strings with category codes in geoCC_16
Replacing strings with category codes in geoCC_17
Replacing strings with category codes in geoCC_18
Replacing strings with category codes in geoCC_19
Replacing 

In [35]:
df.to_pickle(f"dataset_{subset}.pkl.gz", compression='gzip')